In [1]:
from validate_adrresses import validate_address
import pandas as pd
import os
from tqdm import tqdm
print(os.getcwd())

/home/luis/Documents/Belcorp/iteration_adresses


In [2]:
df = pd.read_csv('./data/direcciones.csv', sep=';')

/tmp/ipykernel_3264109/4095483860.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/direcciones.csv', sep=';')


In [3]:
cols = [col for col in df]
col1 = cols[:3]
col2 = cols[3:]

cols = []
for col in col1:
    cols.append(col)
cols.append("NUEVA DIRECCION")
for col in col2:
    cols.append(col)

In [4]:
df["NUEVA DIRECCION"] = None
df = df.reindex(columns=cols)
df = df[df["IND TERRITORIOS"] == 9]

In [5]:
df_tmp = df[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO", "NUEVA DIRECCION", "X", "Y"]]
df_tmp = df_tmp[5000:6000]
df_tmp 

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
24227,FRANCISCO TORNERIA F 005 LOTEO NUEVA CIUDAD II...,CODEGUA,None,,
24228,RIO DE JANEIRO 5633.SAN MIGUEL.METROPOLITANA. ...,PEDRO AGUIRRE CERDA,None,-33.5078939,-70.6419064
24230,STA. TERESA 281 LOS MORROS AV. PADRE HURTADO,SAN BERNARDO,None,,
24243,23 SUR 23 SUR 9 Y MEDIA ORIENTE F 942,MAULE,None,,
24244,LAS CAUCHAS 0 CASA DE UN PISO FORRADA CON ZINC...,BULNES,None,,
...,...,...,...,...,...
28946,TACNA 6320 BA 116 TORRE A EDIF MIRADOR AZUL LA...,LA FLORIDA,None,,
28947,LAUTARO 848 P SAN FRANCISCO,EL MONTE,None,,
28948,"OLIMPO 903, RENCA, CHILE OLIMPO CON AVENIDA SA...",RENCA,None,-33.4468336,-70.6225822
28954,"ESMERALDA 2918, ANTOFAGASTA, CHILE ESMERALDA C...",ANTOFAGASTA,None,,


In [6]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

ddata = dd.from_pandas(df_tmp, npartitions=30)

# Sample meta dataframe to indicate the structure of the output
meta = pd.DataFrame({
    "DIRECCION DE DOMICILIO": pd.Series(dtype='str'),
    "COMUNA DOMICILIO": pd.Series(dtype='str'),
    "NUEVA DIRECCION": pd.Series(dtype='str'),
    "X": pd.Series(dtype='float'),
    "Y": pd.Series(dtype='float')
})

def apply_validate_address(df_chunk):
    # Handle missing values before passing to validate_address
    df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]] = df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]].fillna("")
    
    # Apply the validation function to each row and unpack the results
    df_chunk[["NUEVA DIRECCION", "X", "Y"]] = df_chunk.apply(
        lambda row: pd.Series(validate_address(row["DIRECCION DE DOMICILIO"] + " " + row["COMUNA DOMICILIO"])),
        axis=1
    )
    return df_chunk

# Map the function to each partition, with the meta keyword to define the structure
res = ddata.map_partitions(apply_validate_address, meta=meta).compute()

sin coincidencias ['LAUTARO', 'PALENA']  ------  LAUTARO SN PALENA
['C', '1', 'TRIHUECO', 'VILLA', 'ESPERANZA', 'LOS ALAMOS']  ------  C. 1 TRIHUECO VILLA ESPERANZA LOS ALAMOS  -------  Villa Esperanza, Loncoche, Provincia de Cautín, Región de la Araucanía, 4970348, Chile
['CALLE', '9', 'N°2597', 'VILLA', 'DON', 'VICENTE', 'MOLINA']  ------  CALLE 9 N°2597 VILLA DON VICENTE MOLINA  -------  Calle 9, Don Horacio, Fundo Santa Blanca, Lontué, Molina, Provincia de Curicó, Región del Maule, Chile
['ARTURO', 'PRAT', 'POBLACION', 'GOYCPLEA', 'PASAJE 3', 'CASA 13']  ------  ARTURO PRAT POBLACION GOYCPLEA, PASAJE 3 CASA 13, FRENTE MALL CHINO ANCUD  -------  3, Arturo Prat, Villa Manuel Rengifo, Condominio Rucalhue, Buin, Provincia de Maipo, Región Metropolitana de Santiago, Chile
['GENERAL', 'MANUEL', 'BAQUEDANO', '1168', 'ANTOFAGASTA', 'CHILE', 'ENTRE', 'Y', 'CURICO']  ------  GENERAL MANUEL BAQUEDANO 1168, ANTOFAGASTA, CHILE ENTRE BAQUEDANO Y CURICO ANTOFAGASTA  -------  General Manuel Baqued

In [7]:
res

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
24227,FRANCISCO TORNERIA F 005 LOTEO NUEVA CIUDAD II...,CODEGUA,"Villa Ciudad Nueva Renca II, Renca, Provincia ...",-33.397179,-70.747388
24228,RIO DE JANEIRO 5633.SAN MIGUEL.METROPOLITANA. ...,PEDRO AGUIRRE CERDA,"Río de Janeiro, Población Brasilia, San Miguel...",-33.507703,-70.641774
24230,STA. TERESA 281 LOS MORROS AV. PADRE HURTADO,SAN BERNARDO,"Santa Teresa, Pulmahue, Valle Hermoso, La Ligu...",-32.447041,-71.228009
24243,23 SUR 23 SUR 9 Y MEDIA ORIENTE F 942,MAULE,"23 Sur, Villa Ecuador, La Florida, Talca, Prov...",-35.446857,-71.675351
24244,LAS CAUCHAS 0 CASA DE UN PISO FORRADA CON ZINC...,BULNES,"Camino a Bulnes, Brisas de Palomares, Concepci...",-36.821218,-72.995641
...,...,...,...,...,...
28946,TACNA 6320 BA 116 TORRE A EDIF MIRADOR AZUL LA...,LA FLORIDA,"Calle de Chile, Montealegre, l'Eliana, el Camp...",39.568148,-0.542500
28947,LAUTARO 848 P SAN FRANCISCO,EL MONTE,"Chile, San Juan, Quito, Pichincha, 170405, Ecu...",-0.216831,-78.515746
28948,"OLIMPO 903, RENCA, CHILE OLIMPO CON AVENIDA SA...",RENCA,"Canchas Renca, Avenida Presidente Salvador All...",-33.412142,-70.728415
28954,"ESMERALDA 2918, ANTOFAGASTA, CHILE ESMERALDA C...",ANTOFAGASTA,"Esmeralda, Villa Marina, Penco, Provincia de C...",-36.737564,-72.998638


In [8]:
print("A")

A


In [9]:
res.to_csv('./data/out/ad6.csv')